# Design
This amplifier will be placed between the mixer and the crystal filter.

# Requirements
* Termination insensitive : input impedance that does not depend upon the output load. Further, the output impedance should not depend upon the source at the amplifier input.
* 50 Ohm input and output impedance
* [Signal levels and IF amp gain](https://youtu.be/jwbYQUTUEmE?list=PL9Ox3wpnB0kqekAyz6blg4YdvoEMoJNJY&t=344)

## Common base amplifier followed by a common-collector amplifier
* [A Termination Insensitive Amplifier for Bidirectional Transceivers, W. Hayward, B.Kopski](./doc/bidirectional_matched_amplifier.pdf) shows a 12V common base amplifier followed by a common collector amplifier.  The common base amplifier has a high Ic, so input impedance is very low.  A 47ohm external resistor is used to match the 50 Ohm input impedance.  This divides down the input signal by a large factor.  The CB-amplifier then has to make up for this loss and add 10dB of gain.  So noise figure isn't great.  The first amplifier presented in that paper (CE + CC) has a NF of 6dB.
* [Episode 1 Part 2 in Radio Design 401 : Blocking problem](https://youtu.be/tuOc61TShDg?list=PL9Ox3wpnB0krNexW2k5JMCaewXN7LoRXd&t=736) : a too large amplitude at the base will case Vbe to go too low.  Lower Vbe means lower transconductance, which means lower gain. This is called "blocking" and is a common problem in RF amplifiers.

For a common base amplifier using the MMBT5179 with an Ic of 10mA, maxium vbe amplitude is 1mV for a 0.5% THD (i.e. 2nd harmonic is 4 0dB below the fundamental).

## Common emitter amplifier followed by two common collector amplifiers
<fig>
  <img src="doc/TIA5V_sch.png" alt="missing" width="600"/>
  <figcaption>Common emitter amplifier followed by two common collector amplifiers</figcaption>
</fig>

### Common emitter amplifier design
#### Requirements
* Gain : 15dB
* Frequency : 1MHz -> 100MHz
* Input impedance : 50 Ohm

#### Choice
* Bias current : 7.5mA (as from W. Hayward : "A Termination Insensitive Amplifier for Bidirectional Transceivers")

In [48]:
import math

# Load resistance
VCC = 5.0 # Supply voltage
V_B = 1.36 # Base DC voltage
I_E = 7.5e-3 # Emitter current

# Input impedance
V_T = 26e-3 # Thermal voltage
r_e = V_T / I_E # Internal Emitter resistance

R_f = 330 # Feedback resistor
R_S = 50 # Source resistor
R_L = 220 # Load resistor
R_d = R_S * R_L / (-R_S + R_L + R_f)
r_d = R_d - r_e # External emitter resistance
print(f"External emitter resistance: {r_d:.2f} Ohm")

# Transducer gain
G_t = 4 * R_L * R_S * (R_f - R_d)**2 / (R_d * R_L + R_d * R_f + R_S * R_L + R_S * R_d)**2
G_t_dB = 10 * math.log10(G_t)
print(f"Transducer gain: {G_t_dB:.2f} dB")

External emitter resistance: 18.53 Ohm
Transducer gain: 8.53 dB


In [46]:
# Calculate bias stability
def stability_check(R1, R2, Re, hfe):
    R_th = R1 * R2 / (R1 + R2) # Thevenin resistance

    # Bowick "Transistor Biasing" (ISBN: 978-0-7506-8518-4): As a practical rule of thumb for stable designs, the ratio R_th/Re should be less than 10.
    if(R_th / Re) < 10:
        print("\tStability check: OK")
    else:
        print("\tStability check: NOT OK")

    S = (hfe + 1) * (Re + R_th) / ((hfe+1) * Re + R_th) # Stability factor
    print("\tStability factor: {:.2f}".format(S))
    if S < 10:
        print("\tGood biasing")
    elif S < 20:
        print("\tAcceptable biasing")
    else:
        print("\tUnstable biasing")

# hfe for 2N3904 at 10MHz = 30 (hfe = 100 at DC, 1 at 300MHz)
hfe = 30
stability_check(3.6e3, 1.5e3, 82, hfe)

	Stability check: NOT OK
	Stability factor: 9.82
	Good biasing


### Common collector amplifier design

In [47]:
# Input impedance
R_E = 430
Rin = hfe * R_E
print(f"Input impedance: {Rin:.2f} Ohm")

# Output impedance


Input impedance: 12900.00 Ohm


# Measurements
| Action | Expected Result | Observed result | Status |
|--------|-----------------|-----------------|--------|
| V(C10) | 4.73V           | 4.70V           | ✅   |
| Vb(Q2) | 1.37V           | 1.35V           | ✅   |
| Ve(Q2) | 0.66V           | 0.637V          | ✅   |
| Vb(Q1) | 2.35V           | 2.39V           | ✅   |
| Vb(Q3) | 1.66V           | 1.69V           | ✅   |
| Ve(Q3) | 0.93V           | 0.97V           | ✅   |

<figure>
  <img src="./doc/nanovna-h4_2025-08-11_21-12-49.png" alt="S21 and S11 measurement">
  <figcaption>S-Parameters Measurement</figcaption>
</figure>

# References
* [#207: Basics of a Cascode Amplifier and the Miller Effect](https://www.youtube.com/watch?v=Op_I3Ke7px0)
* [The Cascode Amplifier](http://www.engr.newpaltz.edu/~zunoubm/S20/electronics2/Lecture_Notes/Cascode.pdf)
* EMRFD : §2.7 FEEDBACK AMPLIFIERS
* [Simple SSB Rig: Part 6 - IF Amplifiers](https://www.youtube.com/watch?v=CHdtoupH2Vg) : common emitter, single transistor
* [40m Pelican Case SSB Radio - Part 3: LCD Display, RF Amps Topology](https://youtu.be/0f1rzZ_BglY?list=PLxU5OX0SLgIO4eot9jJvrRcx-NpZgddV-&t=333) : feedback amplifier with 2N3904 (according to EMRFD)
* ARRL Handbook 2018 : §12.4.3 : > 30mA Ic 